In [194]:
import pandas as pd
from PIL import Image
import numpy as np
import json
import os
from cv2 import cv2

font = cv2.FONT_HERSHEY_SIMPLEX
colors = []
for i in range(4):
    for j in range(4):
        for k in range(4):
            colors.append((i*85, j*85, k*85))
            
def get_frame_by_seconds(video_path, seconds):  #提取视频中图片 按照每秒提取 间隔是视频帧率
    vc = cv2.VideoCapture(video_path) #读入视频文件
    fps = vc.get(cv2.CAP_PROP_FPS) #获取帧率
    print(fps) #帧率可能不是整数 需要取整
    rval=vc.isOpened() #判断视频是否打开 返回True或Flase
    c = 0
    over = int(fps*seconds)
    while rval: # 循环读取视频帧
        rval, frame = vc.read() # 第一个返回值为是否成功获取视频帧，第二个返回值为返回的视频帧：
        if rval:
            c = c + 1
            if c==over:
                break
    vc.release()
    return frame

In [201]:
ava_val_v22 = pd.read_csv(r"E:\mmaction2-master\tools\data\ava\annotations\ava_val_v2.2.csv", header=None)
video_root_path = r"E:\mmaction2-master\tools\data\ava\videos/" #视频所在的路径
video_id = '1j20qq1JyX4'
seconds = 902
video_path = video_root_path+'{}.mp4'.format(video_id)
video_sec = ava_val_v22[(ava_val_v22[0]==video_id) & (ava_val_v22[1]==seconds)]
boxes_labels_kinds = get_video_sec[[2, 3, 4, 5, 6, 7]].values
frame = get_frame_by_seconds(video_path, seconds)

with open(r"E:\mmaction2-master\tools\data\ava\annotations\ava_action_list_v2.2.pbtxt",'r') as load_f:
    load_dict = load_f.readlines()
ava_labels = [i.strip()[7: -1] for i in load_dict if 'name' in i]

29.97002997002997


In [198]:
frame_copy = frame.copy()
cv2.imshow('show',frame_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [203]:
frame_copy = frame.copy()
h, w, _ = frame_copy.shape
kinds = boxes_labels_kinds[:, -1]
for kind in kinds:
    boxes_labels_kind = boxes_labels_kinds[kinds==kind][:, :-1]
    x1, y1, x2, y2 = boxes_labels_kind[0, :-1]
    x1, y1, x2, y2 = int(x1*w), int(y1*h), int(x2*w), int(y2*h)
    frame_copy = cv2.rectangle(frame_copy, (x1, y1), (x2, y2), colors[int(kind)%64], 5) 
    for idx, label in enumerate(list(boxes_labels_kind[:, -1])): 
        frame_copy = cv2.putText(frame_copy, ava_labels[int(label-1)], (x1, y1+int(idx*h/30)), font, 0.5, colors[int(label)%64], 1)
    
cv2.imshow('show',frame_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [187]:
import pickle

F=open(r"E:\mmaction2-master\tools\data\ava\annotations\ava_dense_proposals_train.FAIR.recall_93.9.pkl",'rb')
train_content=pickle.load(F)

F=open(r"E:\mmaction2-master\tools\data\ava\annotations\ava_dense_proposals_val.FAIR.recall_93.9.pkl",'rb')
val_content=pickle.load(F)

val_content

{'1j20qq1JyX4,0902': array([[0.036   , 0.098   , 0.55    , 0.979   , 0.995518],
        [0.443   , 0.04    , 0.99    , 0.989   , 0.977824]]),
 '1j20qq1JyX4,0903': array([[0.046   , 0.071   , 0.941   , 0.967   , 0.999744]]),
 '1j20qq1JyX4,0904': array([[0.043   , 0.052   , 0.953   , 0.974   , 0.99967 ],
        [0.821   , 0.456   , 0.986   , 0.931   , 0.050167]]),
 '1j20qq1JyX4,0905': array([[0.205   , 0.021   , 0.988   , 0.976   , 0.999849]]),
 '1j20qq1JyX4,0906': array([[0.202   , 0.027   , 0.986   , 0.977   , 0.999883]]),
 '1j20qq1JyX4,0907': array([[0.029   , 0.128   , 0.958   , 0.963   , 0.999567]]),
 '1j20qq1JyX4,0908': array([[0.015   , 0.018   , 0.301   , 0.854   , 0.164304],
        [0.026   , 0.014   , 0.943   , 0.95    , 0.999305]]),
 '1j20qq1JyX4,0909': array([[0.305   , 0.089   , 0.954   , 0.985   , 0.998075],
        [0.03    , 0.403   , 0.422   , 0.991   , 0.99617 ]]),
 '1j20qq1JyX4,0910': array([[0.05    , 0.033   , 0.823   , 0.98    , 0.999254],
        [0.748   , 0.439

In [202]:
frame_copy = frame.copy()
h, w, _ = frame_copy.shape
kinds = boxes_labels_kinds[:, -1]
for idx, box_score in enumerate(val_content[video_id+','+format(seconds, '04d')]):
    x1, y1, x2, y2, score = box_score
    x1, y1, x2, y2, score = int(x1*w), int(y1*h), int(x2*w), int(y2*h), round(score, 2)
    frame_copy = cv2.rectangle(frame_copy, (x1, y1), (x2, y2), colors[idx%64], 5) 
    frame_copy = cv2.putText(frame_copy, str(score), (x1, y1+int(idx*h/30)), font, 0.5, colors[idx%64], 1)
    
cv2.imshow('show',frame_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
ava数据集固定每一秒只抽出一帧
csv文件是每个视频的每一秒中的一帧的每个人的box以及label作为一行数据标签。
pkl文件是每个视频的每一秒中的一帧作为key，这个key对应的value是这一帧所有的box。

In [ ]:
打标签过程：
    ①选取视频，从视频中每一秒只抽取一帧图片，该图片命名为：该视频名字+第几秒的格式。（用python写代码就可以，基于上面函数：get_frame_by_seconds，改改就可以得到该功能）
    ②利用via对每张图片进行标注，一张图片可以有多个人，一个人可以有多个label，一个人只有一个box。（https://www.bilibili.com/video/BV16Q4y127iT?p=6）
    ③自行编写代码转换，将从via导出的标签文件按照pkl和csv的格式转换。（需要自己编写python代码转换）

In [188]:
ava_labels

['bend/bow (at the waist)',
 'crawl',
 'crouch/kneel',
 'dance',
 'fall down',
 'get up',
 'jump/leap',
 'lie/sleep',
 'martial art',
 'run/jog',
 'sit',
 'stand',
 'swim',
 'walk',
 'answer phone',
 'brush teeth',
 'carry/hold (an object)',
 'catch (an object)',
 'chop',
 'climb (e.g., a mountain)',
 'clink glass',
 'close (e.g., a door, a box)',
 'cook',
 'cut',
 'dig',
 'dress/put on clothing',
 'drink',
 'drive (e.g., a car, a truck)',
 'eat',
 'enter',
 'exit',
 'extract',
 'fishing',
 'hit (an object)',
 'kick (an object)',
 'lift/pick up',
 'listen (e.g., to music)',
 'open (e.g., a window, a car door)',
 'paint',
 'play board game',
 'play musical instrument',
 'play with pets',
 'point to (an object)',
 'press',
 'pull (an object)',
 'push (an object)',
 'put down',
 'read',
 'ride (e.g., a bike, a car, a horse)',
 'row boat',
 'sail boat',
 'shoot',
 'shovel',
 'smoke',
 'stir',
 'take a photo',
 'text on/look at a cellphone',
 'throw',
 'touch (an object)',
 'turn (e.g., a s